In [ ]:
from __future__ import annotations
import base64
import json
import time
from typing import Any, Dict, Iterable, Optional, Union
import requests


API_KEY = "sk-4ubKIPn2VfUuHW151FncRQ"
API_ENDPOINT = "https://api.thucchien.ai/chat/completions"
BUDGET_USD= 50

class APIClient:
    def __init__(self) -> None:
        self.headers: Dict[str, str] = {"Content-Type": "application/json"}
        if API_KEY:
            self.headers["Authorization"] = f"Bearer {API_KEY}"
        self.total_cost = 0.0
    def call_text_api(
        self, prompt: str, is_json_output: bool = True
    ) -> Union[Dict[str, Any], str]:
        model_name = "gemini-2.5-pro"
        payload = {
            "model": model_name,
            "messages": [{"role": "user", "content": prompt}],
            "response_format": {"type": "json_object"}
        }
        print(f"--- [API Call] Calling Text Model: {model_name} ---")
        response_data = self._make_request(payload)
        try:
            content_str = self._extract_text_content(response_data)
            return content_str
        except (json.JSONDecodeError, KeyError, IndexError, ValueError) as e:
            print(
                f"!!! Error parsing JSON from API response: {e}\nRaw content: {response_data.get('choices')}"
            )
        raise ValueError("Failed to parse JSON from API response.")
    def call_image_api(self, prompt: str) -> bytes:
        model_name = "image-gen-4"
        payload = {
            "model": model_name,
            "prompt": prompt,
            "response_format": "b64_json",
            "size": "1920x1080",
        } 
        print(f"--- [API Call] Calling Image Model: {model_name} ---")
        response_data = self._make_request(payload)
        try:
            data_list = response_data.get("data") or []
            if not data_list:
                raise KeyError("Missing 'data' in response")
            b64_string = data_list[0]["b64_json"]
            return base64.b64decode(b64_string)
        except (KeyError, IndexError, TypeError) as e:
            print(f"!!! Error parsing image data: {e}")
            raise ValueError("Failed to get base64 image from API response.")
    def call_tts_api(self, text: str, voice_style: str) -> bytes:
        model_name = "text-to-speech-hq"
        payload = {"model": model_name, "input": text, "voice": voice_style}
        print(
            f"--- [API Call] Calling TTS Model: {model_name} ---"
        )  # Giả định API TTS trả về file audio trực tiếp
        response = requests.post(
            API_ENDPOINT, headers=self.headers, json=payload, timeout=60
        )
        response.raise_for_status()
        # Logic theo dõi chi phí cho TTS
        cost = (len(text) / 1000) * COSTS[model_name]["per_1k_chars"]
        self.total_cost += cost
        print(
            f"        Cost for this call: ${cost:.4f} | Total cumulative cost: ${self.total_cost:.4f} / ${BUDGET_USD}"
        )
        return response.content
    def _make_request(
        self, payload: Dict[str, Any], retries: int = 3
    ) -> Dict[str, Any]:
        for attempt in range(retries):
            try:
                start_time = time.time()
                response = requests.post(
                    API_ENDPOINT, headers=self.headers, json=payload, timeout=120
                )
                response.raise_for_status()
                end_time = time.time()
                print(f"--- API Call successful ({end_time - start_time:.2f}s) ---")
                return response.text
            except requests.exceptions.RequestException as e:
                print(f"!!! API Error (Attempt {attempt + 1}/{retries}): {e}")
                if attempt == retries - 1:
                    raise
            time.sleep(2**attempt)  # Exponential backoff
        return {}
    @staticmethod
    def _extract_text_content(response_data: Dict[str, Any]) -> str:
        choices = response_data.get("choices")
        if not choices:
            raise ValueError("No 'choices' in response.")
        message = choices[0].get("message", {})
        content: Optional[Union[str, Iterable[Dict[str, Any]]]] = message.get("content")
        if isinstance(content, str):
            return content
        if isinstance(content, list):
            text_chunks = [
                chunk.get("text", "")
                for chunk in content
                if isinstance(chunk, dict) and chunk.get("type") == "text"
            ]
            combined = "\n".join(chunk for chunk in text_chunks if chunk)
            if combined:
                return combined
        # Fallback to legacy format
        if "content" in message and isinstance(message["content"], dict):
            return json.dumps(message["content"])
        raise ValueError("Unsupported response format from text API.")
api_client = APIClient()

In [ ]:
style = {'art_style_description': "Phong cách nghệ thuật là sự giao thoa giữa 'Flat 2.0' và 'Anime Chibi' hiện đại, với cảm hứng từ thế giới 'ăn vặt' đường phố đầy màu sắc của Việt Nam. Nhân vật và linh vật được thiết kế theo phong cách Chibi đáng yêu, mắt to tròn, biểu cảm phong phú để tạo sự gần gũi và kết nối cảm xúc với người chơi. Môi trường game (cổng trường, gánh hàng rong) được tái hiện qua các mảng khối vector tối giản, sử dụng bảng màu tươi sáng và hiệu ứng gradient nhẹ nhàng để tạo chiều sâu. Các món ăn được vẽ cách điệu, nhấn mạnh vào đặc điểm nhận dạng (màu sắc, bao bì), giúp người chơi dễ dàng phân biệt thực phẩm 'An Toàn' (rực rỡ, tươi mới) và 'Nguy Hiểm' (màu sắc u tối, có dấu hiệu lạ). Giao diện người dùng (UI) sạch sẽ, icon minh hoạ trực quan, và typography sử dụng font chữ không chân, bo tròn, mang lại cảm giác thân thiện, dễ đọc và hiện đại.",
 'color_palette': [{'name': 'Xanh Lá Mạ (Tươi Sạch)',
   'hex': '#88D8B0',
   'description': "Màu chủ đạo cho các yếu tố 'an toàn', 'sạch'. Tượng trưng cho sự tươi mới, thiên nhiên và sức khỏe. Dùng cho các câu trả lời đúng, thực phẩm an toàn, và các thông điệp tích cực."},
  {'name': 'Vàng Nghệ (Năng Lượng)',
   'hex': '#FFC145',
   'description': 'Màu sắc thương hiệu chính, mang lại cảm giác vui tươi, năng động và hấp dẫn. Dùng cho các nút kêu gọi hành động (CTA) chính, điểm số, và các phần thưởng trong game.'},
  {'name': 'Cam Cà Rốt (Cảnh Báo Thân Thiện)',
   'hex': '#FF6B6B',
   'description': 'Màu nhấn mạnh, thu hút sự chú ý. Sử dụng cho các thông báo quan trọng, hướng dẫn, hoặc các yếu tố tương tác cần người chơi tập trung.'},
  {'name': 'Hồng Dâu (Ngọt Ngào)',
   'hex': '#F7CAD9',
   'description': 'Màu sắc phụ trợ, tạo cảm giác vui nhộn, ngọt ngào, gần gũi với các món ăn vặt. Dùng cho các chi tiết trang trí, icon phụ, và các hiệu ứng ăn mừng.'},
  {'name': 'Xanh Chàm (Tin Tưởng)',
   'hex': '#3A506B',
   'description': 'Màu chữ chính và các khối thông tin kiến thức. Tạo độ tương phản cao, dễ đọc, mang lại cảm giác đáng tin cậy và có tính giáo dục.'},
  {'name': 'Xám Khói (Nguy Hiểm)',
   'hex': '#9B9B9B',
   'description': "Đại diện cho các yếu tố 'không an toàn', hàng giả, thực phẩm bẩn. Dùng cho các lựa chọn sai, các vật phẩm nguy hiểm, và các thông điệp cảnh báo tiêu cực."},
  {'name': 'Trắng Kem (Nền Tảng)',
   'hex': '#F8F9FA',
   'description': 'Màu nền chủ đạo, tạo không gian sạch sẽ, thoáng đãng, làm nổi bật các màu sắc khác và giúp người chơi tập trung vào nội dung chính.'}]}

In [7]:
api_client.call_text_api(f"""
Bạn là một Kiến trúc sư Thông tin và Biên tập viên Nội dung. Dựa trên chủ đề
"Giáo dục nhận thức về an toàn thực phẩm, tập trung vào kỹ năng nhận biết thực
phẩm sạch/an toàn và phân biệt hàng thật/hàng giả (đặc biệt là thực phẩm tại cổng trường)"
an toàn thực phẩm" và Style Guide sau: {style}.

Sản phẩm: Xây dựng một game tương tác (Mobile Web Game).
Đối tượng: Học sinh Trung học cơ sở (THCS).
Chủ đề: Giáo dục nhận thức về an toàn thực phẩm, tập trung vào kỹ năng nhận biết hàng thật/hàng giả (đặc biệt là thực phẩm tại cổng trường).
Yêu cầu chính:
Thiết kế một nhân vật mascot (linh vật) giả tưởng, xuyên suốt trò chơi.
Gameplay: Thiết kế sinh động, vui nhộn, dễ chơi.
Nội dung: Sử dụng các tình huống minh họa bằng hình ảnh (đồ họa do AI tạo) để giúp trẻ nhận diện và ghi nhớ kiến thức.
Sản phẩm đầu ra: Game (dạng mobile web) được triển khai trên một tên miền công khai (public domain) để mọi người có thể truy cập và chơi thử.
Yêu cầu về tính sáng tạo: Sản phẩm phải là sáng tạo mới 100%, đáp ứng đầy đủ yêu cầu đề bài, tuân thủ quy định pháp lý và chuẩn mực giáo dục. Không sao chép hoặc mô phỏng từ sản phẩm có sẵn.
Ngôn ngữ: Tiếng Việt.
BỐI CẢNH
Game sẽ được ứng dụng rộng rãi trong các chiến dịch truyền thông về vệ sinh an toàn thực phẩm tại trường học, các sự kiện cộng đồng, và lan tỏa trên các kênh truyền thông của ngành giáo dục (trang web trường, fanpage...).
NỘI DUNG
Giải quyết vấn đề nhức nhối về tình trạng thực phẩm không an toàn, hàng giả, hàng nhái được bán quanh khu vực cổng trường học.
Thay vì các phương pháp tuyên truyền truyền thống, việc ứng dụng game tương tác giúp học sinh chủ động hơn trong việc tiếp thu kiến thức, từ đó tăng hiệu quả truyền thông và giáo dục.


Thực hiện tuần tự các bước suy luận sau (Chain-of-Thought):
1. Xác định 4 câu chuyện cốt lõi cần truyền tải, tương ứng với 4 trang: Trang
chủ (Tổng quan), Dòng thời gian Lịch sử, Không khí Ngày hội Toàn quốc, và Tầm
vóc Việt Nam (Tương lai).
2. Với mỗi trang, thiết kế các "sections" (khối nội dung) logic. Mỗi section
phải có "type" (ví dụ: 'hero', 'timeline', 'gallery', 'article', 'quote') và một
"content_brief" mô tả ngắn gọn nội dung cần có.
3. Với mỗi section, hãy viết nội dung văn bản (content) chi tiết, súc tích, văn
phong tiếng Việt chuẩn, trang trọng.
4. Với mỗi section cần hình ảnh, hãy viết một "image_brief" mô tả ý tưởng
hình ảnh minh họa theo đúng `art_style_description`.
5. Với mỗi trang, hãy tạo một ÿối tượng "seo" chứa "meta_title" và
"meta_description" phù hợp.
Trả về dưới dạng một đối tượng JSON duy nhất, gồm 5 màn chơi trách nghiệm đúng sai không giải thích gì thêm,
tuân thủ nghiêm ngặt cấu trúc sau:
{{
    "question": "Câu hỏi dựa trên hình ảnh",
    "true_image": "Nội dung hình ảnh hàng thật",
    "false_image": "Nội dung hình ảnh hàng giả"
}}
""")

--- [API Call] Calling Text Model: gemini-2.5-pro ---
--- API Call successful (29.93s) ---


[{'question': "Gói bim bim nào là 'hàng xịn' đáng tin cậy để bạn chọn?",
  'true_image': 'Phong cách Flat 2.0 & Chibi. Một gói bim bim màu Vàng Nghệ (#FFC145) rực rỡ. Bao bì căng phồng, sạch sẽ. Chữ in và logo thương hiệu sắc nét, có ghi rõ hạn sử dụng bằng font chữ bo tròn trên nền màu Xanh Chàm (#3A506B). Bên cạnh là một nhân vật Chibi đang vui vẻ giơ ngón tay cái tán thành.',
  'false_image': 'Phong cách Flat 2.0 & Chibi. Một gói bim bim có màu Xám Khói (#9B9B9B) u tối. Bao bì nhăn nhúm, có vết bẩn. Chữ in bị nhòe, sai lỗi chính tả, không có thông tin hạn sử dụng. Nhân vật Chibi bên cạnh đang cau mày, lắc đầu tỏ vẻ lo lắng.'},
 {'question': 'Chai nước ngọt nào an toàn để giải khát dưới trời nắng?',
  'true_image': 'Phong cách Flat 2.0 & Chibi. Một chai nước có thiết kế hiện đại, nắp chai còn nguyên niêm phong. Nhãn dán thẳng thớm, màu sắc tươi tắn (Xanh Lá Mạ #88D8B0), thông tin in rõ ràng. Nước bên trong có màu trong veo, hấp dẫn. Nhân vật Chibi mắt to tròn, hào hứng chỉ vào chai n

In [14]:
api_client.call_text_api("""
Hãy tạo 1 game mobile web trắc nghiệm với style guide như sau:

{'art_style_description': "Phong cách nghệ thuật là sự giao thoa giữa 'Flat 2.0' và 'Anime Chibi' hiện đại, với cảm hứng từ thế giới 'ăn vặt' đường phố đầy màu sắc của Việt Nam. Nhân vật và linh vật được thiết kế theo phong cách Chibi đáng yêu, mắt to tròn, biểu cảm phong phú để tạo sự gần gũi và kết nối cảm xúc với người chơi. Môi trường game (cổng trường, gánh hàng rong) được tái hiện qua các mảng khối vector tối giản, sử dụng bảng màu tươi sáng và hiệu ứng gradient nhẹ nhàng để tạo chiều sâu. Các món ăn được vẽ cách điệu, nhấn mạnh vào đặc điểm nhận dạng (màu sắc, bao bì), giúp người chơi dễ dàng phân biệt thực phẩm 'An Toàn' (rực rỡ, tươi mới) và 'Nguy Hiểm' (màu sắc u tối, có dấu hiệu lạ). Giao diện người dùng (UI) sạch sẽ, icon minh hoạ trực quan, và typography sử dụng font chữ không chân, bo tròn, mang lại cảm giác thân thiện, dễ đọc và hiện đại.",
 'color_palette': [{'name': 'Xanh Lá Mạ (Tươi Sạch)',
   'hex': '#88D8B0',
   'description': "Màu chủ đạo cho các yếu tố 'an toàn', 'sạch'. Tượng trưng cho sự tươi mới, thiên nhiên và sức khỏe. Dùng cho các câu trả lời đúng, thực phẩm an toàn, và các thông điệp tích cực."},
  {'name': 'Vàng Nghệ (Năng Lượng)',
   'hex': '#FFC145',
   'description': 'Màu sắc thương hiệu chính, mang lại cảm giác vui tươi, năng động và hấp dẫn. Dùng cho các nút kêu gọi hành động (CTA) chính, điểm số, và các phần thưởng trong game.'},
  {'name': 'Cam Cà Rốt (Cảnh Báo Thân Thiện)',
   'hex': '#FF6B6B',
   'description': 'Màu nhấn mạnh, thu hút sự chú ý. Sử dụng cho các thông báo quan trọng, hướng dẫn, hoặc các yếu tố tương tác cần người chơi tập trung.'},
  {'name': 'Hồng Dâu (Ngọt Ngào)',
   'hex': '#F7CAD9',
   'description': 'Màu sắc phụ trợ, tạo cảm giác vui nhộn, ngọt ngào, gần gũi với các món ăn vặt. Dùng cho các chi tiết trang trí, icon phụ, và các hiệu ứng ăn mừng.'},
  {'name': 'Xanh Chàm (Tin Tưởng)',
   'hex': '#3A506B',
   'description': 'Màu chữ chính và các khối thông tin kiến thức. Tạo độ tương phản cao, dễ đọc, mang lại cảm giác đáng tin cậy và có tính giáo dục.'},
  {'name': 'Xám Khói (Nguy Hiểm)',
   'hex': '#9B9B9B',
   'description': "Đại diện cho các yếu tố 'không an toàn', hàng giả, thực phẩm bẩn. Dùng cho các lựa chọn sai, các vật phẩm nguy hiểm, và các thông điệp cảnh báo tiêu cực."},
  {'name': 'Trắng Kem (Nền Tảng)',
   'hex': '#F8F9FA',
   'description': 'Màu nền chủ đạo, tạo không gian sạch sẽ, thoáng đãng, làm nổi bật các màu sắc khác và giúp người chơi tập trung vào nội dung chính.'}]}

Web site sử dụng Javascript, HTML và CSS với cấu trúc như sau:

- assets/
  - mascot.png
  - questions/
     - question-1/
         - true.png
         - false.png
     - question-2/
         - true.png
         - false.png
     ...
     - question-10/
         - true.png
         - false.png
- index.html
- styles.css
- main.js
- README.md
- vercel.json

Game sẽ gồm menu, hướng dẫn, cài đặt độ khó: với dễ sẽ gồm 4 câu hỏi, trung bình sẽ là 7 câu hỏi, và khó sẽ là 10 câu hỏi. Cùng với đó là màn hình chiến thắng, thất bại cùng với chơi lại.

Hãy viết các file index.html, styles.css, main.js, README.md, vercel.json.
""")

--- [API Call] Calling Text Model: gemini-2.5-pro ---
--- API Call successful (0.32s) ---


AttributeError: 'str' object has no attribute 'get'